In [8]:
!pip install ctgan

In [9]:
import pandas as pd
import numpy as np
from ctgan import CTGAN
from sklearn.preprocessing import MinMaxScaler

## Why CTGAN for synthetic data generation?

CTGAN is suitable for our synthetic data generation because it can effectively manage mixed data types (binary, categorical, and continuous) present in our dataset. Its ability to capture complex relationships between variables ensures that the synthetic data maintains the statistical properties of the original records. Additionally, CTGAN allows for conditional generation, which can help create tailored samples based on specific features, enhancing the relevance and realism of the synthetic data we produce.

In [10]:
#contains all 36 metrics as well as the soldier name which is kept soldier_0, soldier_1 etc since we do not know their language and how'd they would name their people.
def create_empty_betrayal_metrics_df():
    columns = [
        'soldier_name','familial_history','dependents', 'kin', 'pay_gap', 'risk_last_op', 'active_duty_record', 'served_time', 'off_time', 'current_posting', 'posting_period', 'failure','corruption', 'physical_health', 'injury_type', 'mental_health', 'campaign_count', 'campaign_cause', 'peer_based', 'grievance', 'n_reports', 'complaint_c', 'absents', 'task_sf', 'genb_location', 'genc_location', 'age', 'edu_level', 'previously_c', 'attack','riots','emergency','x_incentives','p_incentives','risk','cultural_change','discipline'
    ]
    df = pd.DataFrame(columns=columns)
    return df

In [11]:
betrayal_df = create_empty_betrayal_metrics_df()

In [12]:
#division of columns as per their values it can contain
binary_columns = [
    'campaign_cause','complaint_c','task_sf','attack','riots','emergency','cultural_change'
]

continuous_columns = [
    'risk', 'discipline','edu_level','mental_health','risk_last_op','active_duty_record','failure'
]

integer_columns = list(set(betrayal_df.columns) - set(binary_columns) - set(continuous_columns) - {'soldier_name'})

In [13]:
variable_ranges = {
    # Continuous variables with 0-1 range
    'risk': (0, 1),
    'risk_last_op': (0, 1),
    'mental_health': (0, 1),
    'edu_level': (0, 1),
    'discipline': (0, 1),
    'active_duty_record': (0, 1),
    'failure': (0, 1),

    # Integer variables with specific ranges
    'served_time': (0, 50),
    'familial_history': (0, 30),
    'off_time': (0, 1000),
    'current_posting': (0, 11),
    'posting_period': (0, 15),
    'corruption': (0, 10000),
    'physical_health': (0, 1000),
    'injury_type': (0, 3),
    'campaign_count': (0, 50),
    'peer_based': (0, 100),
    'grievance': (0, 1000),
    'n_reports': (0, 100),
    'absents': (0, 100),
    'kin': (0, 20),
    'dependents': (0, 20),
    'pay_gap': (0, 10000),
    'genb_location': (0, 500),
    'genc_location': (0, 500),
    'age': (18, 60),
    'previously_c': (0, 100),
    'x_incentives': (0, 10000),
    'p_incentives': (0, 10000)
}


In [14]:
def generate_random_data(n_samples=1000):
    data = []
    for _ in range(n_samples):
        sample = {}
        #binary columns: generate 0 or 1
        for col in binary_columns:
            sample[col] = np.random.choice([0, 1])

        #continuous columns: generate a float between the range 0 and 1
        for col in continuous_columns:
            sample[col] = np.random.uniform(0, 1)

        #integer columns: generate an integer within the specified range
        for col in integer_columns:
            sample[col] = np.random.randint(variable_ranges[col][0], variable_ranges[col][1] + 1)

        sample['soldier_name'] = 'placeholder_name'

        data.append(sample)

    return pd.DataFrame(data)

In [15]:
initial_data = generate_random_data(n_samples=1000)

df_train = initial_data.drop(columns=['soldier_name'])

In [16]:
#normalising the data
scaler = MinMaxScaler()
df_train_normalized = pd.DataFrame(scaler.fit_transform(df_train), columns=df_train.columns)

In [17]:
ctgan = CTGAN(epochs=500)
ctgan.fit(df_train_normalized)

In [18]:
synthetic_data_normalized = ctgan.sample(500)

Denormalizing in CTGAN is essential for the following reasons since

synthetic data generated by CTGAN is often normalized to improve training stability. Denormalizing the data restores it to its original scale, making it easier to interpret and use in practical applications

Ensures that the generated data accurately reflects the actual value ranges and distributions of the original dataset, enhancing its realism and usability.


In [19]:
#denormalising the data
synthetic_data = pd.DataFrame(scaler.inverse_transform(synthetic_data_normalized), columns=df_train.columns)

a common challenge when using generative models like CTGAN is that they learn the overall distribution of the data but don't inherently understand hard constraints like minimum and maximum values.

so we apply post-processing to the data

In [20]:
def clip_to_range(data, ranges):
    for col, (min_val, max_val) in ranges.items():
        if col in data.columns:
            data[col] = data[col].clip(min_val, max_val)
    return data

for col in binary_columns:
    synthetic_data[col] = synthetic_data[col].round().astype(int)

synthetic_data = clip_to_range(synthetic_data, variable_ranges)

for col in integer_columns:
    if col in synthetic_data.columns:
        synthetic_data[col] = synthetic_data[col].round().astype(int)


In [21]:
synthetic_data['soldier_name'] = ['soldier_' + str(i) for i in range(len(synthetic_data))]

In [22]:
#verification of adherence to ranges
for col, (min_val, max_val) in variable_ranges.items():
    if col in synthetic_data.columns:
        actual_min = synthetic_data[col].min()
        actual_max = synthetic_data[col].max()
        if actual_min < min_val or actual_max > max_val:
            print(f"Warning: {col} has values outside the specified range. Min: {actual_min}, Max: {actual_max}")

In [23]:
!pip install skimpy

  Using cached jedi-0.19.1-py2.py3-none-any.whl.metadata (22 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.2/117.2 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 28.8/28.8 MB 30.4 MB/s eta 0:00:00
Using cached jedi-0.19.1-py2.py3-none-any.whl (1.6 MB)
  Attempting uninstall: typeguard
    Found existing installation: typeguard 4.3.0
    Uninstalling typeguard-4.3.0:
      Successfully uninstalled typeguard-4.3.0
  Attempting uninstall: polars
    Found existing installation: polars 1.6.0
    Uninstalling polars-1.6.0:
      Successfully uninstalled polars-1.6.0
  Attempting uninstall: ipykernel
    Found existing installation: ipykernel 5.5.6
    Uninstalling ipykernel-5.5.6:
      Successfully uninstalled ipykernel-5.5.6
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires ipykernel==5.5.6, but you have ipykern

In [24]:
import skimpy as skim

skim.skim(synthetic_data)

╭──────────────────────────────────────────────── skimpy summary ─────────────────────────────────────────────────╮
│          Data Summary                Data Types                                                                 │
│ ┏━━━━━━━━━━━━━━━━━━━┳━━━━━━━━┓ ┏━━━━━━━━━━━━━┳━━━━━━━┓                                                          │
│ ┃ dataframe         ┃ Values ┃ ┃ Column Type ┃ Count ┃                                                          │
│ ┡━━━━━━━━━━━━━━━━━━━╇━━━━━━━━┩ ┡━━━━━━━━━━━━━╇━━━━━━━┩                                                          │
│ │ Number of rows    │ 500    │ │ int64       │ 29    │                                                          │
│ │ Number of columns │ 37     │ │ float64     │ 7     │                                                          │
│ └───────────────────┴────────┘ │ string      │ 1     │                                                          │
│                                └─────────────┴───────┘                                                          │
│                                                     number                                                      │
│ ┏━━━━━━━━━━━━━━━━━━━━━━┳━━━━━┳━━━━━━━┳━━━━━━━━━┳━━━━━━━━━┳━━━━━┳━━━━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━┳━━━━━━━━┓  │
│ ┃ column_name          ┃ NA  ┃ NA %  ┃ mean    ┃ sd      ┃ p0  ┃ p25      ┃ p50    ┃ p75    ┃ p100  ┃ hist   ┃  │
│ ┡━━━━━━━━━━━━━━━━━━━━━━╇━━━━━╇━━━━━━━╇━━━━━━━━━╇━━━━━━━━━╇━━━━━╇━━━━━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━╇━━━━━━━━┩  │
│ │ campaign_cause       │   0 │     0 │   0.398 │    0.49 │   0 │        0 │      0 │      1 │     1 │ ▇    ▅ │  │
│ │ complaint_c          │   0 │     0 │   0.516 │  0.5002 │   0 │        0 │      1 │      1 │     1 │ ▇    ▇ │  │
│ │ task_sf              │   0 │     0 │    0.45 │   0.498 │   0 │        0 │      0 │      1 │     1 │ ▇    ▇ │  │
│ │ attack               │   0 │     0 │   0.718 │  0.4504 │   0 │        0 │      1 │      1 │     1 │ ▃    ▇ │  │
│ │ riots                │   0 │     0 │   0.368 │  0.4827 │   0 │        0 │      0 │      1 │     1 │ ▇    ▅ │  │
│ │ emergency            │   0 │     0 │   0.658 │  0.4749 │   0 │        0 │      1 │      1 │     1 │ ▅    ▇ │  │
│ │ cultural_change      │   0 │     0 │   0.304 │  0.4604 │   0 │        0 │      0 │      1 │     1 │ ▇    ▃ │  │
│ │ risk                 │   0 │     0 │  0.5078 │  0.3287 │   0 │   0.1999 │ 0.5686 │ 0.7905 │     1 │ ▇▅▃▅▇▇ │  │
│ │ discipline           │   0 │     0 │  0.6028 │  0.3144 │   0 │   0.3392 │ 0.6579 │   0.89 │     1 │ ▃▃▃▃▅▇ │  │
│ │ edu_level            │   0 │     0 │  0.3676 │  0.3149 │   0 │  0.08836 │ 0.2784 │ 0.6536 │     1 │ ▇▅▃▂▃▂ │  │
│ │ mental_health        │   0 │     0 │  0.4358 │  0.3182 │   0 │   0.1533 │ 0.3835 │ 0.7089 │     1 │ ▇▆▃▅▅▅ │  │
│ │ risk_last_op         │   0 │     0 │  0.3564 │   0.274 │   0 │    0.106 │   0.32 │ 0.5823 │     1 │ ▇▅▅▅▃▁ │  │
│ │ active_duty_record   │   0 │     0 │  0.3791 │  0.3262 │   0 │  0.03087 │ 0.3253 │ 0.6579 │     1 │ ▇▃▁▃▃▂ │  │
│ │ failure              │   0 │     0 │  0.6521 │  0.2997 │   0 │   0.3851 │ 0.7068 │ 0.9368 │     1 │ ▂▃▃▃▃▇ │  │
│ │ x_incentives         │   0 │     0 │    4138 │    3085 │   0 │     1504 │   3358 │   6882 │ 10000 │ ▇▆▃▃▃▃ │  │
│ │ campaign_count       │   0 │     0 │   18.05 │   15.34 │   0 │        3 │     15 │     30 │    50 │ ▇▃▂▃▂▂ │  │
│ │ corruption           │   0 │     0 │    6057 │    3191 │   0 │     3538 │   6518 │   8968 │ 10000 │ ▃▃▃▃▅▇ │  │
│ │ familial_history     │   0 │     0 │   16.94 │   9.687 │   0 │        9 │     17 │     26 │    30 │ ▃▃▅▃▅▇ │  │
│ │ grievance            │   0 │     0 │   412.8 │   281.8 │   0 │      176 │    378 │  653.5 │  1000 │ ▇▇▅▆▆▃ │  │
│ │ posting_period       │   0 │     0 │   5.352 │   4.552 │   0 │        1 │      4 │      9 │    15 │ ▇▃▂▃▃▁ │  │
│ │ age                  │   0 │     0 │   41.84 │   12.77 │  18 │       31 │     44 │     53 │    60 │ ▃▅▃▅▅▇ │  │
│ │ dependents           │   0 │     0 │   7.476 │   6.1

In [25]:
#saving the synthetic data to a CSV file
synthetic_data.to_csv('soldiers_Xernia.csv', index=False)
print("Synthetic data has been saved to 'soldiers_Xernia.csv'")

Synthetic data has been saved to 'soldiers_Xernia.csv'
